**Method 01**

In [0]:
data = [15, 25, 36, 44, 57, 65, 89, 95, 9]

df4 = spark.createDataFrame([(x,) for x in data], ["Numbers"])
display(df4)

Numbers
15
25
36
44
57
65
89
95
9


**✅ Why use (x,) and not just [x for x in data]?**

- **[(x,) for x in data]** creates a **list of 1-element tuples**:

      [(15,), (25,), (36,), ..., (9,)]

  - Each item is a **tuple with one element**, which maps correctly to the **one-column schema ["Numbers"]**. PySpark sees **each tuple as a row**.
  
  - **createDataFrame()** expects each element of the input **list to be a row**, which means a **tuple or list** representing the values in each column.

- So, when you write:

      [(x,) for x in data]

  - **[(x,)]** creates a **list of tuples**, each containing a **single value** — this is what PySpark expects when you give it a schema like ["Numbers"]

  - **each row** has **1 value** (for the column Numbers), so create it as a **tuple of one element**.

In [0]:
df4 = spark.createDataFrame([x for x in data], ["Numbers"])
display(df4)

---------------------------------------------------------------------------
PySparkTypeError                          Traceback (most recent call last)
File <command-3931534879136477>, line 1
----> 1 df4 = spark.createDataFrame([x for x in data], ["Numbers"])
      2 display(df4)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1610, in SparkSession.createDataFrame(self, data, schema, samplingRatio, verifySchema)
   1605 if has_pandas and isinstance(data, pd.DataFrame):
   1606     # Create a DataFrame from pandas DataFrame.
   1607     return super(SparkSession, self).createDataFrame(  # type: ignore[call-overload

**❌ What happens with [x for x in data]?**

- **[x for x in data]** produces

      [15, 25, 36, 44, 57, 65, 89, 95, 9]

  - This is a **list of integers, not a list of rows/tuples**.
  - These are **integers, not tuples or rows**. Spark doesn't know how to treat an **int as a row**. It expects something like **(15,) or [15]** to match with the column definition **["Numbers"]**.
- If you try:

      df = spark.createDataFrame([15, 25, 36], ["Numbers"])

- You'll get an **error** like:

      TypeError: StructType can only be created from list or tuple, got <class 'int'>
      TypeError: StructType can not be applied to an int

  - Because Spark tries to interpret **15 as a row**, and it **can't match** it to the **schema**.

**✅ Method 02: Alternate using List**

- This works too, because each **[x]** is a **one-element list**, Spark can treat this as a **single-row** entry.

In [0]:
df4 = spark.createDataFrame([[x] for x in data], ["Numbers"])
display(df4)

Numbers
15
25
36
44
57
65
89
95
9


**✅ Method 03: Alternative using Row**

In [0]:
from pyspark.sql import Row

df = spark.createDataFrame([Row(Numbers=x) for x in data])
display(df)

Numbers
15
25
36
44
57
65
89
95
9


**Summary:**
| Expression	| Works?	 | Reason |
|-------------|----------|--------|
| [(x,) for x in data]	| ✅	| Tuple per row (1-element row). Spark accepts this.|
| [[x] for x in data]	| ✅	| List per row. Spark accepts this as well. |
| [x for x in data]	| ❌	| Just a list of integers — Spark can't treat plain ints as row data. |

**Method 04: List of tuples (single element)**

In [0]:
# Create a sample DataFrame
data = [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]
df = spark.createDataFrame(data, ["id"])
display(df)

id
1
2
3
4
5
6
7
8
9
10
